<a href="https://colab.research.google.com/github/pretty-neat-org/steve-translator/blob/main/yolo_detection/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Apr 12 10:26:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive


Mounted at /content/gdrive


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14751 (delta 0), reused 1 (delta 0), pack-reused 14748
Receiving objects: 100% (14751/14751), 13.31 MiB | 20.65 MiB/s, done.
Resolving deltas: 100% (10031/10031), done.


In [ ]:
#!rm -rf darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "text" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-12 10:28:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  60.6MB/s    in 2.6s    

2021-04-12 10:28:37 (60.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/slika1.jpg     
  inflating: data/obj/slika1.txt     
  inflating: data/obj/slika2.jpg     
  inflating: data/obj/slika2.txt     
  inflating: data/obj/slika3.jpg     
  inflating: data/obj/slika3.txt     
  inflating: data/obj/slika4.jpg     
  inflating: data/obj/slika4.txt     
  inflating: data/obj/slika5.jpg     
  inflating: data/obj/slika5.txt     
  inflating: data/obj/slika6.jpg     
  inflating: data/obj/slika6.txt     
  inflating: data/obj/slika7.jpg     
  inflating: data/obj/slika7.txt     
  inflating: data/obj/slika8.jpg     
  inflating: data/obj/slika8.txt     
  inflating: data/obj/slika9.jpg     
  inflating: data/obj/slika9.txt     
  inflating: data/obj/slika10.jpg    
  inflating: data/obj/slika10.txt    
  inflating: data/obj/slika11.jpg    
  inflating: data/obj/slika11.txt    
  inflating: data/obj/slika12.jpg    
  inflating: data/obj/slika12.txt    
  inflating: data/obj/slika13.jpg    
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re
import cv2

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:
                # Definisanje klase i koordinata objekata za svaki objekat iz txt fajla
                # format je <object-class> <x> <y> <width> <height> relativno u odnosu na 
                # koordinate slike

                xmin = int(numbers[0])
                ymin = int(numbers[1])
                xmax = int(numbers[2])
                ymax = int(numbers[3])

                bbox_height = ymax - ymin
                bbox_width = xmax - xmin

                x = xmin + int(bbox_width/2)
                y = ymin + int(bbox_height/2)
                
                img_path = file_path.split(".")[0]+".jpg"
                im = cv2.imread(img_path)
                im_height = im.shape[0]
                im_width = im.shape[1]

                text = "0 {} {} {} {}".format(x/im_width, y/im_height, bbox_width/im_width, bbox_height/im_height)
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Upis u fajl
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
67 424 625 524 "Overseas"

['67', '424', '625', '524']
218 data/obj/slika133.txt
0 0.2703125 0.49375 0.4359375 0.10416666666666667
723 434 1253 531 "Relations"

['723', '434', '1253', '531']
218 data/obj/slika133.txt
0 0.771875 0.5020833333333333 0.4140625 0.10104166666666667
488 594 850 693 "Office"

['488', '594', '850', '693']
218 data/obj/slika133.txt
0 0.52265625 0.6697916666666667 0.2828125 0.103125
584, 399, 679, 420, "Metool"

['584', '399', '679', '420']
219 data/obj/slika291.txt
0 0.49296875 0.42604166666666665 0.07421875 0.021875
97 217 457 309 "Nightline"

['97', '217', '457', '309']
220 data/obj/slika151.txt
0 0.4328125 0.5479166666666667 0.5625 0.19166666666666668
1495, 546, 1697, 604, "Teer:"

['1495', '546', '1697', '604']
221 data/obj/slika267.txt
0 0.6157407407407407 0.1478909465020576 0.0779320987654321 0.014917695473251029
1505, 624, 1547, 683, "7"

['1505', '624', '1547', '683', '7']
221 data/obj/slika267.txt
0 0.5

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")

In [ ]:
#Pravljenje training.txt fajla
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Obucavanje mreze
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.816975), count: 4, class_loss = 0.214116, iou_loss = 0.124308, total_loss = 0.338424 
 total_bbox = 260769, rewritten_bbox = 0.047935 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.879129), count: 1, class_loss = 0.231927, iou_loss = 0.021245, total_loss = 0.253172 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.864744), count: 3, class_loss = 0.162778, iou_loss = 0.036414, total_loss = 0.199192 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.724832), count: 7, class_loss = 0.944256, iou_loss = 0.452087, total_loss = 1.396343 
 total_bbox = 260780, rewritten_bbox = 0.047933 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.681054), count: 3, class_loss = 0.577899, iou_loss = 0.152833, total_loss = 0.730732 
v3 (m